## House Price prediction Project with MLFLOW tracking

In [1]:
import pandas as pd  
import mlflow  
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.datasets import fetch_california_housing

## DATA PREPARATION

In [2]:
housing=fetch_california_housing()

print(housing)

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]], shape=(20640, 8)), 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894], shape=(20640,)), 'frame': None, 'target_names': ['MedHouseVal'], 'feature_names': ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude'], 'DESCR': '.. _california_housing_dataset

In [3]:
housing.values()

dict_values([array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]], shape=(20640, 8)), array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894], shape=(20640,)), None, ['MedHouseVal'], ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude'], '.. _california_housing_dataset:\n\nCalifornia Housing dataset\n------------------------

In [4]:
housing.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])

In [5]:
## preparing the data
df=pd.DataFrame(housing.data, columns=housing.feature_names)

df['Price']=housing.target

df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


## Train Test Split,Hyper parameter Tuning,Tracking Experiments Through MLFLOW 

In [6]:
from urllib.parse import urlparse

In [7]:
## dividing the data into Independent and Dependent features
X=df.drop('Price', axis=1)
y=df['Price']

In [8]:
X.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [9]:
y.head()

0    4.526
1    3.585
2    3.521
3    3.413
4    3.422
Name: Price, dtype: float64

In [25]:
## Hyperparameter tuning using GridSearchCV

def hyperparameter_tuning(X_train,y_train,param_grid):
    rf=RandomForestRegressor()
    grid_search=GridSearchCV(estimator=rf,param_grid=param_grid,cv=2,n_jobs=-1,verbose=2,scoring='neg_mean_squared_error')
    grid_search.fit(X_train,y_train)
    return grid_search

In [ ]:
## Split the data into training and testing
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

from mlflow.models import infer_signature

signature=infer_signature(X_train, RandomForestRegressor().fit(X_train, y_train).predict(X_train))

## Define the hyperparameter grid 
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5,10,None],
    'min_samples_split': [2,5],
    'min_samples_leaf': [1,2]
}

## start the mlflow experiment
with mlflow.start_run():
    grid_search=hyperparameter_tuning(X_train,y_train,param_grid)

    ## Get the best model
    best_model=grid_search.best_estimator_

    ## Evaluate the best model
    y_pred=best_model.predict(X_test)

    mse=mean_squared_error(y_test,y_pred)

    ## Log the best parameters and metrics
    mlflow.log_param("best_n_estimators", grid_search.best_params_["n_estimators"])

    mlflow.log_param("best_max_depth", grid_search.best_params_["max_depth"])

    mlflow.log_param("best_min_samples_split", grid_search.best_params_["min_samples_split"])
    
    mlflow.log_param("best_min_samples_leaf", grid_search.best_params_["min_samples_leaf"])



    ## Tracking uri 
    mlflow.set_tracking_uri("http://127.0.0.1:5000")
    tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme

    if tracking_url_type_store != "file":
        mlflow.sklearn.log_model(best_model,"best-model",registered_model_name="best RandomForestRegressor Model")
    else:
        mlflow.sklearn.log_model(best_model,"model",signature=signature)


print(f"Best Hyperparameters: {grid_search.best_params_}")
print(f"mean_squared_error: {mse}")


Fitting 2 folds for each of 24 candidates, totalling 48 fits


2025/03/04 02:35:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'best RandomForestRegressor Model'.
2025/03/04 02:35:17 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: best RandomForestRegressor Model, version 1
Created version '1' of model 'best RandomForestRegressor Model'.
2025/03/04 02:35:17 INFO mlflow.tracking._tracking_service.client: 🏃 View run intelligent-kit-261 at: http://127.0.0.1:5000/#/experiments/0/runs/09cd9b6ef05246e7b21ff3bc4464fb5b.
2025/03/04 02:35:17 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0.


Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
mean_squared_error: 0.2526815079647757
